<a href="https://colab.research.google.com/github/DSlaughter01/election-stability/blob/main/electoral_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A cleaner attempt at creating an election class

In [ ]:
# Import and install necessary modules
!pip install igraph
!pip install cairocffi

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import igraph as ig
import cairocffi
from statistics import mode
from collections import Counter
from math import floor, ceil
from heapq import nlargest
from random import choices, choice
import seaborn as sns
import time
import warnings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
class Election():

  # Which functions do we need?
  def __init__(self, n = 500, part = 3, elecs = 100, iters = 5000, dist = 10,
               pref = 1, zeal = 0.05, alpha = 0.05, t = 0, track = False):

    self.n = n          # The number of nodes in the network
    self.part = part    # The number of parties available to vote for
    self.elecs = elecs  # The number of elections to perform
    self.iters = iters  # Number of interactions between voters
    self.dist = dist    # Number of electoral districts (= seats available for now)
    self.pref = pref    # Number of voter party preferences
    self.zeal = zeal    # Probability that a node in the zealot party is a zealot
    self.alpha = alpha  # Noise rate
    self.t = t          # Whether it is the first election or not
    self.track = track  # Whether tracking of every iteration is desired

    # The final number of votes and seats in each election - to be
    self.votes = np.empty((self.part, self.elecs))
    self.seats = np.empty((self.part, self.elecs))

    # Tracking votes over iterations of the voter model if desired
    self.vm_tracker = []
    if self.track == True:
      self.vm_tracker = np.empty((self.elecs, self.iters, self.part))

    # Check parameter values
    if self.zeal > 1: raise ValueError("Please enter a zeal value between 0 and 1")
    if self.alpha > 1: raise ValueError("Please enter an alpha value between 0 and 1")



  def init_graph(self):
    # Function to initialise a graph, and allocate zealots and opinions

    a = self.dist

    # First generate an SBM
    # The preference matrix - symmetric, stronger on the diagonal, random
    lt = np.tril(np.random.rand(10, 10), k = -1)
    ut = np.transpose(lt)
    diags = np.diag(np.random.rand(10))
    pref_mat = 0.25*(lt + ut + diags) + 0.75*(np.identity(10))

    # Generate the SBM from this preference matrix
    split = np.array_split(range(self.n), a)
    block_sizes = [len(i) for i in split]
    start_graph = ig.Graph.SBM(n = self.n,
                               pref_matrix = list(pref_mat),
                               block_sizes = block_sizes)




  # One to perform iterations of the voter model
  def voter_model(self, track = False):
    pass

  # One to plot iterations of the voter model
  def vm_plot(self):
    pass

  # One for each electoral system
  # FPTP
  def fptp(self):
    pass

  # PR
  def pr(self):
    pass

  # Two-round system - may be difficult allocating political opinions
  def two_round(self):
    pass

  # STV - need to allocate multiple opinions
  def stv(self):
    pass

  # One to plot how the underlying network looks
  def net_plot(self):
    pass

  # One to plot histograms
  def plot_hist(self):
    pass



  # How can we efficiently allocate space, and reduce time?

In [ ]:
g = Election()
g.init_graph()